In [1]:
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal


In [2]:
# Set up the matrices 
Phi1 = np.array([[0.5, -0.9, 0], 
                 [1.1, -0.7, 0], 
                 [0, 0, 0.5]])
Phi2 = np.array([[0, -0.5, 0],
                 [-0.5, 0, 0],
                 [0, 0, 0]])

Theta1 = np.array([[0.4, 0.8, 0],
                   [-1.1, -0.3, 0],
                   [0, 0, 0]])
Theta2 = np.array([[0, -0.8, 0],
                   [-1.1, 0, 0],
                   [0, 0, 0]])

# Define the covariance matrix for U
sigma_U = np.zeros((3, 3))
for i in range(3):
    for j in range(3):
        sigma_U[i, j] = ((-1)**(i+j) * 0.05)


In [3]:
#Use NumPy to generate multivariate normal innovations.
def generate_innovations(n_periods):
    return multivariate_normal.rvs(mean=np.zeros(3), cov=sigma_U, size=n_periods)


In [4]:
#Initialize the process and simulate over a given number of periods.
def simulate_arma22(n_periods):
    X = np.zeros((n_periods, 3))
    U = generate_innovations(n_periods)

    for t in range(2, n_periods):
        X[t] = Phi1 @ X[t-1] + Phi2 @ X[t-2] + U[t] + Theta1 @ U[t-1] + Theta2 @ U[t-2]
    return X


In [5]:
A = 2.5 * np.eye(3)
B = 7.5 * np.array([[0, -1, -1],
                    [-1, 0, -1],
                    [-1, -1, 0]])

def generate_demand(X, epsilon):
    delta = np.random.normal(0, 1, size=(len(X), 3)) / 4  # Independent noise
    Y = np.maximum(0, X @ A.T + (X + delta) @ B.T + epsilon)
    return Y


In [6]:
n_periods = 100
X = simulate_arma22(n_periods)
epsilon = np.random.normal(0, 1, (n_periods, 3))
Y = generate_demand(X, epsilon)


In [8]:
df = pd.DataFrame(Y, columns=['Demand1', 'Demand2', 'Demand3'])
print(df)


     Demand1   Demand2   Demand3
0   0.000000  0.000000  0.000000
1   1.432684  3.046718  0.679892
2   0.000000  2.084631  0.000000
3   1.484592  2.658390  4.355172
4   0.286238  2.750635  2.447508
..       ...       ...       ...
95  0.000000  0.000000  0.000000
96  0.000000  0.000000  4.911172
97  0.000000  0.000000  0.000000
98  1.726504  0.000000  5.133561
99  0.000000  1.539247  0.000000

[100 rows x 3 columns]
